In [2]:
import numpy as np

# generate data

In [75]:
def generate_data(dim, num):
    x = np.random.normal(0, 10, [num, dim])
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

In [76]:
# example
x, y, mr = generate_data(5, 100)

In [79]:
x[:5], y[:5]

(array([[ -0.12389315,  -6.64142401,   6.31900201,  26.835902  ,
          -5.29100515],
        [ -1.24327523,  -4.81021143,   7.13429738,   5.70732553,
          12.37027471],
        [ 13.80831041, -11.87690428,  14.30611638,  -0.19146284,
           9.49275913],
        [  7.01726142,   0.7902737 ,  -7.42342723,  -1.47241787,
          -3.06668564],
        [  0.36858018,   9.34200794,   5.34787713,  -6.68833832,
         -18.67568807]]),
 array([[-1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.]]))

# write your model class

In [ ]:
# you can do anything necessary about the model
class SVM1:
    def __init__(self, dim):
        pass

    def fit(self, X, y):
        """
        Fit the coefficients via your methods
        """
        
    def predict(self, X):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """

In [ ]:
# you can do anything necessary about the model
class SVM2:
    def __init__(self, dim):
        pass

    def fit(self, X, y):
        """
        Fit the coefficients via your methods
        """
        
    def predict(self, X):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """

# construct and train your models

# predict and compare your results